<a href="https://colab.research.google.com/github/MohammedNasserAhmed/MHIP/blob/master/anvil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

APP URL :
https://health-cover-estimate.anvil.app/

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=b0f223c3ee6769d22ac57f033aefe86728773a45b8000cd416c68a1cb3448a00
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = joblib.load('/content/gdrive/MyDrive/Colab_Notebooks/Medical Health Insurance Cost Prediction /rfr__for_mhip_98.pkl')
scaler = joblib.load('/content/gdrive/MyDrive/Colab_Notebooks/Medical Health Insurance Cost Prediction /scaler.pkl')


In [ ]:
import anvil.server
anvil.server.connect("server_D5NI56R664SNVMHMBSM2WJTT-OCB7BLIZBBZFCQ2Y")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
def scale_data(values):
    column_names = ['age', 'bmi', 'children', 'sex', 'smoker']

    # Create a dictionary to map column names to list elements
    data = dict(zip(column_names, values))
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame([data])
    df.iloc[:,:3]=scaler.transform(df.iloc[:,:3])
    format_function = lambda x: f'{x:.2f}'

    # Apply the formatting function to all values in the DataFrame
    df = df.applymap(format_function)
    return df

In [ ]:
@anvil.server.callable
def predict_charges(age, bmi, children, sex_male, smoker_yes):
  data = scale_data(values=[age, bmi, children, sex_male, smoker_yes])
  data = data.iloc[0].values
  predicted_chrages = model.predict([data])
  return "{:.2f} $".format(predicted_chrages[0])

In [ ]:
anvil.server.wait_forever()